In [1]:
from data_selection import (
    PrepareData,
    TrainValTestSplit,
    TimeSeriesAnalyzer
)

___
# Preparar series de tiempo

In [2]:
prepare_data = PrepareData(
    path_data="data.csv", 
    colname_datetime="date",
    colname_features=["sales_u_day","sales_usd_day","promo"],
    colname_id_time_series=["id_product","id_distributor"]
)

In [3]:
prepare_data()

Archivo cargado desde /mnt/atrys/Desafios/data.csv
Se encontraron 1000 series de tiempo con id ID_ts.
Datos desde 2019-04-08 hasta 2019-12-31, (268 dias) 
Generando series de tiempo


100%|██████████| 1000/1000 [00:06<00:00, 143.73it/s]


In [4]:
prepare_data.save()

Guardando series de tiempo


100%|██████████| 1000/1000 [00:00<00:00, 4883.58it/s]

Series de tiempo guardadas en /mnt/atrys/DL-Pipelines/ECG/Supervised_Learning_Forecasting/time_series


___
# Elegir conjuntos de train, val y test

In [5]:
import json
with open("time_series/time_series_config.json", "r", encoding="utf8") as fp:
    ts_config = json.load(fp)

In [6]:
ts_config.keys()

dict_keys(['features', 'timesteps', 'id_time_series', 'basepath_time_series', 'list_min', 'list_max', 'list_mean', 'list_std'])

In [7]:
inputs_split = dict(
    id_time_series = ts_config.get("id_time_series"),
    timesteps = ts_config.get("timesteps"),
    features = ts_config.get("features"),
    basepath_time_series = ts_config.get("basepath_time_series"),
)

tvt_split = TrainValTestSplit(
    **inputs_split
)

In [8]:
# Definir porcentaje de entrenamiento (lo restante se divide a la mitad en val y test)
# Definir largo de la entrada y de la salida (este ultimo fijo a 1 para este problema)
ds = tvt_split(perc_train = 0.7, len_input=30, len_output=1)

Conjuntos de train, val y test creados. Ver atributos .datasets


In [9]:
tvt_split.save()

Guardando conjuntos train, val, test y labels
Datos para entrenamiento guardados en /mnt/atrys/DL-Pipelines/ECG/Supervised_Learning_Forecasting/data


___
# Remover datos que no cumplen con ciertos filtros
- porción maxima de la serie con datos nulos
- presencia de features completamente nulas
- norma de alguna de las features es menor a eps_norm (para emular lo anterior pero en datos ruidosos)
- features que tengan un único valor (constante, igual a un vector nulo para efectos del entrenamiento)

In [10]:
from model_training import LoadTimeSerie
from data_selection import TimeSeriesAnalyzer
from model_training import Preprocessing
from model_training.utils import (
    fromJSON, 
    asJSON
)

from tqdm.notebook import tqdm # barra de progreso

subtract_mean registered
minmax registered
znormalization registered


In [11]:
ts_loader = LoadTimeSerie() # cargar serie de tiempo a numpy (X,y)
ts_analyzer = TimeSeriesAnalyzer(
    max_zero_timesteps=0.7,  # porcion maxima de la serie con datos nulos
    non_zero_features=True,  # rechazar series con al menos una feature nula
    eps_norm=0.00001,        # rechazar si la norma de alguna de las features es menor a eps_norm  
    features_with_unique_values=False # features con valores unicos (como el vector 'is_promo')
)  

# Preprocesamiento
ts_config = fromJSON("time_series/time_series_config.json")
list_min = ts_config.get("list_min")
list_max = ts_config.get("list_max")
list_std = ts_config.get("list_std")
list_mean = ts_config.get("list_mean")

# Las funciones que se pueden usar estan en 'model_training/preprocessing.py'
preprocessing = Preprocessing([
    ("znormalization", dict(axis_signal=0, list_mean=list_mean, list_std=list_std ))
])

In [12]:
# Leer id_ts y analizarlas
list_train = fromJSON("data/list_train.json")
list_val = fromJSON("data/list_val.json")
list_test = fromJSON("data/list_test.json")
labels = fromJSON("data/labels.json")

def get_clean_list(list_id_ts, labels):
    "Analizar una lista de series de tiempo y retornas solo las que pasan el filtro"
    list_clean = []
    for id_ts in tqdm(list_id_ts):
        X,y = ts_loader(idx_feature_y=0 , **labels.get(id_ts))
        
        # Aplicar el filtro
        if ts_analyzer(preprocessing(X)):
#         if ts_analyzer(X):
            list_clean.append(id_ts)
    return list_clean

In [13]:
new_list_train = get_clean_list(list_train, labels)
new_list_val = get_clean_list(list_val, labels)
new_list_test = get_clean_list(list_test, labels)

  0%|          | 0/157000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/11000 [00:00<?, ?it/s]

In [14]:
diff_train = len(list_train) - len(new_list_train)
diff_val = len(list_val) - len(new_list_val)
diff_test = len(list_test) - len(new_list_test)

In [15]:
print(f"""
Se removieron
{diff_train} de entrenamiento. Antes {len(list_train)} | Ahora {len(new_list_train)}
{diff_val} de validacion. Antes {len(list_val)} | Ahora {len(new_list_val)}
{diff_test} de testeo. Antes {len(list_test)} | Ahora {len(new_list_test)}
""")


Se removieron
69035 de entrenamiento. Antes 157000 | Ahora 87965
4889 de validacion. Antes 10000 | Ahora 5111
7552 de testeo. Antes 11000 | Ahora 3448



In [16]:
asJSON(new_list_train, "data/list_train.json", sort_keys=False)
asJSON(new_list_val, "data/list_val.json", sort_keys=False)
asJSON(new_list_test, "data/list_test.json", sort_keys=False)